## AssemblyAI Challenge
by Nura Kawa

Instructions for the challenge: https://docs.google.com/document/d/14nddyffRNvMIu4PDRy3oPqMMPYi-dkW_a8-c0RXQaXk/edit?tab=t.0

### Setup

In [1]:
# If needed, run the line below
# !pip install assemblyai

### Step 2: Transcribe an audio file

In [1]:
import assemblyai as aai

aai.settings.api_key = "" # place your API key here

config = aai.TranscriptionConfig(
    speaker_labels=True,
    speakers_expected=2,
    sentiment_analysis=True)

transcriber = aai.Transcriber()
transcript = transcriber.transcribe(
  "https://assembly.ai/weaviate-podcast-109.mp3",
  config=config
)

Calculate and print the number of different sentences.

In [2]:
sentences = transcript.get_sentences()
n_sentences = len(sentences)

print(f'The transcript has {n_sentences} sentences.')

# Note: I have gotten results in the range [361, 364].

The transcript has 362 sentences.


#### Step 3: Identify Speakers and Analyze Sentiments

1) Identify speakers in the podcast.

In [5]:
for utterance in transcript.utterances[0:5]:
    print(f'{utterance.speaker}: {utterance.text}')

A: Hey everyone, thank you so much for watching another episode of the weva podcast. I'm super excited to welcome Erica Cardenas. Erica is a technology partner manager at weaviate, where she leads the effort behind weaviate recipes and all sorts of things managing weaviate's partnerships with partners such as Google Cohere, Modal and many others. Eric and I, as well as other members from the Weaviate team, will be in New York City next Tuesday, November 19th at the Google Pier 57 building. We'd love to meet you there, so if interested in that, please see the link in the description. Erica, thanks so much for joining the WEV8 podcast.
B: Awesome. Thank you for having me. I'm excited to talk about Agentic Rag and hopefully meet a few people at the event in New York.
A: Awesome. So for our listeners who aren't aware, last week Eric and Leigh and I published a new blog post on weaviate's blogs, Agentic Rag. And so in this podcast we really want to just keep diving into Agentic Rag, explain

In [6]:
# I am not successful in extracting Speaker names. Luckily, we can ask an LLM to help us!

speaker_prompt = "What are the names of the two speakers in this podcast?"
speaker_prompt_result = transcript.lemur.task(
    speaker_prompt, final_model=aai.LemurModel.claude3_5_sonnet
)

In [7]:
print(speaker_prompt_result.response)

The two speakers in this podcast are:

1. Connor (the host)
2. Erica Cardenas

Connor introduces Erica at the beginning of the podcast as a technology partner manager at Weaviate. They have a conversation throughout the podcast about topics related to AI agents, RAG (Retrieval-Augmented Generation), and generative feedback loops.


In [8]:
change_name = lambda x: 'Speaker A (Connor)' if x == 'A' else 'Speaker B (Erica)'    

for utterance in transcript.utterances:
    utterance.speaker = change_name(utterance.speaker)

2. Print the transcript by speaker utterances:


In [9]:
for utterance in transcript.utterances:
    print(f'{utterance.speaker}: {utterance.text}')

Speaker A (Connor): Hey everyone, thank you so much for watching another episode of the weva podcast. I'm super excited to welcome Erica Cardenas. Erica is a technology partner manager at weaviate, where she leads the effort behind weaviate recipes and all sorts of things managing weaviate's partnerships with partners such as Google Cohere, Modal and many others. Eric and I, as well as other members from the Weaviate team, will be in New York City next Tuesday, November 19th at the Google Pier 57 building. We'd love to meet you there, so if interested in that, please see the link in the description. Erica, thanks so much for joining the WEV8 podcast.
Speaker B (Erica): Awesome. Thank you for having me. I'm excited to talk about Agentic Rag and hopefully meet a few people at the event in New York.
Speaker A (Connor): Awesome. So for our listeners who aren't aware, last week Eric and Leigh and I published a new blog post on weaviate's blogs, Agentic Rag. And so in this podcast we really 

3. Apply Sentiment Analysis, and calculate and print the number of POSITIVE, NEUTRAL, and NEGATIVE sentiments



In [10]:
n_positive, n_neutral, n_negative = 0, 0, 0

for sentiment_result in transcript.sentiment_analysis:
    if sentiment_result.sentiment == 'POSITIVE':
       n_positive += 1
    elif sentiment_result.sentiment == 'NEUTRAL':
       n_neutral += 1
    elif sentiment_result.sentiment == 'NEGATIVE':
       n_negative += 1
    else:
       next

print(f'The transcript contains {n_positive} positive, {n_neutral} neutral, and {n_negative} negative utterances.')
 

The transcript contains 75 positive, 269 neutral, and 16 negative utterances.


#### Step 4: Use an LLM to summarize and answer a question

In [11]:
# Summarize with LLM

prompt = "Please provide a short, bullet-point summary of the transcript for a layperson."

result = transcript.lemur.task(
    prompt, final_model=aai.LemurModel.claude3_5_sonnet
)

print(result.response)

Here's a brief, bullet-point summary of the transcript for a layperson:

• The podcast discusses "Agentic RAG" (Retrieval-Augmented Generation), which is an advanced AI system.

• Agents in AI are systems that can use language models, memory, planning, and tools to complete tasks.

• Agentic RAG differs from regular RAG by being able to plan steps, use multiple tools, and search different sources of information.

• The speakers discuss various aspects of AI agents, including:
  - Planning and reasoning capabilities
  - Multi-agent systems where different AI agents work together
  - Memory management in AI systems
  - Challenges in evaluating AI performance

• They explore how Weaviate (a company) is using AI agents in their work, particularly with "generative feedback loops."

• The conversation touches on the future of AI development, including:
  - Long-running AI processes for complex tasks
  - The need for human oversight in AI systems
  - Potential improvements in AI tools and use

In [13]:
# Answer a question

prompt_question = "Based on the transcript, what is agentic RAG?"

prompt_question_result = transcript.lemur.task(
    prompt_question, final_model=aai.LemurModel.claude3_5_sonnet
)

print(prompt_question_result.response)

Based on the transcript, agentic RAG (Retrieval-Augmented Generation) differs from vanilla RAG in the following key ways:

1. Planning ability: Agentic RAG can plan the required steps needed to achieve the user's query, breaking down complex questions into sub-questions.

2. Multiple tool usage: It can call and utilize multiple tools or data sources (e.g., database queries, web searches, APIs) to gather information, rather than just retrieving from a single source.

3. Iterative searching: Agentic RAG can determine if it needs more information and retrieve from additional sources iteratively.

4. Advanced querying: It can navigate databases more effectively, using vector search, aggregations, and filters based on the query requirements.

5. Autonomy: Agentic RAG has more autonomy to query different databases and sources of information as needed.

6. Reasoning: It can use chain-of-thought reasoning or other planning frameworks (like REACT) to better understand and address the user's que

## Thank you!